<a href="https://colab.research.google.com/github/DJCordhose/ml-workshop/blob/master/notebooks/process/flask-server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Serving using Flask

Links
* making flask server directly run on Colab: https://stackoverflow.com/questions/54465816/how-to-use-flask-in-google-colaboratory-python-notebook

In [4]:
!ls -l

total 4
drwxr-xr-x 1 root root 4096 Jul  3 16:14 sample_data


In [3]:
import pickle

# loading models only once on startup
model = pickle.load(open('models/dt.model', 'rb'))

y_le = pickle.load(open('models/y_le.model', 'rb'))
speed_std_scale = pickle.load(open('models/speed_std_scale.model', 'rb'))
age_std_scale = pickle.load(open('models/age_std_scale.model', 'rb'))
miles_std_scale = pickle.load(open('models/miles_std_scale.model', 'rb'))

def predict(speed, age, miles):
    sample = [[speed_std_scale.transform([speed])[0],
               age_std_scale.transform([age])[0],
               miles_std_scale.transform([miles])[0]]]

    result = model.predict(sample)
    result_group = y_le.inverse_transform(result)[0]
    prediction = model.predict_proba(sample)[0].tolist()
    
    return result_group, prediction, y_le.classes_.tolist()

FileNotFoundError: ignored

In [1]:
from flask import Flask, request, jsonify
import socket
print(socket.gethostbyname(socket.getfqdn(socket.gethostname())))

app = Flask(__name__)

@app.route("/ping")
def ping():
    return "pong"

@app.route('/', methods=['GET', 'POST'])
def do_predict():
    speed = request.json['speed']
    age = request.json['age']
    miles = request.json['miles']

    predicted_category, probabilities, classes = predict(speed, age, miles)
    # print ('predicted_category:', predicted_category)
    # print ('prediction:', prediction)

    response = {
        'category': predicted_category,
        'prediction': probabilities,
        'classes': classes
    }
    return jsonify(response)
  
import threading
threading.Thread(target=app.run, kwargs={'host': '0.0.0.0', 'port': 80}).start()  

172.28.0.2
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [2]:
import requests
r = requests.get("http://172.28.0.2/ping")
print(r.status_code)
print(r.encoding)
print(r.apparent_encoding)
print(r.text)

 * Running on http://0.0.0.0:80/ (Press CTRL+C to quit)
172.28.0.2 - - [22/Jul/2019 14:10:29] "GET /ping HTTP/1.1" 200 -


200
utf-8
ascii
pong
